# Libraries and Dependencies

In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Bokeh visualization library
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (ColumnDataSource, HoverTool, Select, CustomJS, 
                         LinearColorMapper, ColorBar)
from bokeh.layouts import column, row
from bokeh.transform import factor_cmap, transform
from bokeh.palettes import Category20, Viridis256

# Data Cleaning and Normalization
import unicodedata
import re

# Initialize Bokeh for Jupyter notebook
output_notebook()

Loading BokehJS ...

# Data Processing

## Preprocessing Functions

In [11]:
# Create mapping dictionaries for common city name variations
city_mappings = {
    'BOGOTA': 'BOGOTA',
    'BOGOTA DC': 'BOGOTA',
    'BOGOTA D.C.': 'BOGOTA',
    'SANTA FE DE BOGOTA': 'BOGOTA',
    'MEDELLIN': 'MEDELLIN',
    'ANTIOQUIA': 'ANTIOQUIA',
    'VALLE DEL CAUCA': 'VALLE DEL CAUCA',
    'VALLE': 'VALLE DEL CAUCA',
    'CUNDINAMARCA': 'CUNDINAMARCA',
    'SANTANDER': 'SANTANDER',
    'NORTE DE SANTANDER': 'NORTE DE SANTANDER',
    'BOYACA': 'BOYACA',
    'VILLA DE LEYVA': 'VILLA DE LEYVA',
    'RAQUIRA': 'RAQUIRA',
    'MONIQUIRA': 'MONIQUIRA',
    'CHIQUINQUIRA': 'CHIQUINQUIRA',
    'SACHICA': 'SACHICA',
    'SUTAMARCHAN': 'SUTAMARCHAN',
    'TUNJA': 'TUNJA',
    'PAIPA': 'PAIPA',
    'DUITAMA': 'DUITAMA',
    'SOGAMOSO': 'SOGAMOSO',
    'NOBSA': 'NOBSA',
    'TIBASOSA': 'TIBASOSA',
    'AQUITANIA': 'AQUITANIA',
    'MONGUI': 'MONGUI',
    'IZA': 'IZA',
    'TOTA': 'TOTA',
    'CUITIVA': 'CUITIVA',
    'FIRAVITOBA': 'FIRAVITOBA',
    'COMBITA': 'COMBITA',
    'VENTAQUEMADA': 'VENTAQUEMADA',
    'SAMACA': 'SAMACA',
    'CUCAITA': 'CUCAITA',
    'MOTAVITA': 'MOTAVITA',
    'OICATA': 'OICATA',
    'TOCA': 'TOCA',
    'SORACA': 'SORACA',
    'SORA': 'SORA',
    'CHIVATA': 'CHIVATA',
    'SIACHOQUE': 'SIACHOQUE',
    'SANTA SOFIA': 'SANTA SOFIA'
}

# Create mapping dictionaries for travel motivation variations
travel_motivation_mappings = {
    # Tourism variations
    'TURISMO': 'TURISMO',
    'TURISTICO': 'TURISMO',
    'TURISTICA': 'TURISMO',
    'RECREACION': 'TURISMO',
    'RECREATIVO': 'TURISMO',
    'VACACIONES': 'TURISMO',
    'DESCANSO': 'TURISMO',
    'OCIO': 'TURISMO',
    'PLACER': 'TURISMO',
    'ENTRETENIMIENTO': 'TURISMO',
    
    # Business variations
    'NEGOCIOS': 'NEGOCIOS',
    'TRABAJO': 'NEGOCIOS',
    'LABORAL': 'NEGOCIOS',
    'EMPRESARIAL': 'NEGOCIOS',
    'COMERCIAL': 'NEGOCIOS',
    'PROFESIONAL': 'NEGOCIOS',
    'REUNION': 'NEGOCIOS',
    'REUNIONES': 'NEGOCIOS',
    'CONFERENCIA': 'NEGOCIOS',
    'CAPACITACION': 'NEGOCIOS',
    
    # Study/Education variations
    'ESTUDIO': 'EDUCACION',
    'ESTUDIOS': 'EDUCACION',
    'EDUCACION': 'EDUCACION',
    'EDUCATIVO': 'EDUCACION',
    'ACADEMICO': 'EDUCACION',
    'UNIVERSIDAD': 'EDUCACION',
    'CURSO': 'EDUCACION',
    'FORMACION': 'EDUCACION',
    'CAPACITACION': 'EDUCACION',
    'INVESTIGACION': 'EDUCACION',
    
    # Health variations
    'SALUD': 'SALUD',
    'MEDICO': 'SALUD',
    'MEDICINA': 'SALUD',
    'TRATAMIENTO': 'SALUD',
    'HOSPITAL': 'SALUD',
    'CLINICA': 'SALUD',
    'TERAPIA': 'SALUD',
    'REHABILITACION': 'SALUD',
    
    # Family/Personal variations
    'FAMILIA': 'PERSONAL/FAMILIAR',
    'FAMILIAR': 'PERSONAL/FAMILIAR',
    'PERSONAL': 'PERSONAL/FAMILIAR',
    'VISITA': 'PERSONAL/FAMILIAR',
    'VISITAR': 'PERSONAL/FAMILIAR',
    'AMIGOS': 'PERSONAL/FAMILIAR',
    'PARIENTES': 'PERSONAL/FAMILIAR',
    'MATRIMONIO': 'PERSONAL/FAMILIAR',
    'BODA': 'PERSONAL/FAMILIAR',
    'FUNERAL': 'PERSONAL/FAMILIAR',
    'CELEBRACION': 'PERSONAL/FAMILIAR',
    
    # Religious variations
    'RELIGION': 'RELIGIOSO',
    'RELIGIOSO': 'RELIGIOSO',
    'RELIGIOSA': 'RELIGIOSO',
    'PEREGRINACION': 'RELIGIOSO',
    'ESPIRITUAL': 'RELIGIOSO',
    'IGLESIA': 'RELIGIOSO',
    'MISA': 'RELIGIOSO',
    'CULTO': 'RELIGIOSO',
    
    # Transit variations
    'TRANSITO': 'TRANSITO',
    'PASO': 'TRANSITO',
    'CONEXION': 'TRANSITO',
    'ESCALA': 'TRANSITO',
    'TRANSFERENCIA': 'TRANSITO',
    
    # Other variations
    'OTRO': 'OTROS',
    'OTROS': 'OTROS',
    'OTRA': 'OTROS',
    'DIFERENTES': 'OTROS',
    'VARIOS': 'OTROS',
    'MULTIPLE': 'OTROS',
    'NS/NR': 'NO ESPECIFICA',
    'NO SABE': 'NO ESPECIFICA',
    'NO RESPONDE': 'NO ESPECIFICA',
    'SIN ESPECIFICAR': 'NO ESPECIFICA'
}

# Create mapping dictionaries for transportation mode variations
transportation_mappings = {
    # Private vehicle variations
    'AUTOMOVIL': 'VEHICULO PARTICULAR',
    'AUTO': 'VEHICULO PARTICULAR',
    'CARRO': 'VEHICULO PARTICULAR',
    'VEHICULO PROPIO': 'VEHICULO PARTICULAR',
    'VEHICULO PARTICULAR': 'VEHICULO PARTICULAR',
    'COCHE': 'VEHICULO PARTICULAR',
    'VEHICULO PRIVADO': 'VEHICULO PARTICULAR',
    'CAMIONETA': 'VEHICULO PARTICULAR',
    'CAMIONETA PROPIA': 'VEHICULO PARTICULAR',
    'PICK UP': 'VEHICULO PARTICULAR',
    
    # Bus variations
    'BUS': 'AUTOBUS',
    'AUTOBUS': 'AUTOBUS',
    'OMNIBUS': 'AUTOBUS',
    'COLECTIVO': 'AUTOBUS',
    'BUSETA': 'AUTOBUS',
    'MICROBUS': 'AUTOBUS',
    'BUS INTERMUNICIPAL': 'AUTOBUS',
    'BUS URBANO': 'AUTOBUS',
    'TRANSPORTE PUBLICO': 'AUTOBUS',
    'SERVICIO PUBLICO': 'AUTOBUS',
    
    # Motorcycle variations
    'MOTOCICLETA': 'MOTOCICLETA',
    'MOTO': 'MOTOCICLETA',
    'MOTOCICLO': 'MOTOCICLETA',
    'CICLOMOTOR': 'MOTOCICLETA',
    'SCOOTER': 'MOTOCICLETA',
    
    # Bicycle variations
    'BICICLETA': 'BICICLETA',
    'BICI': 'BICICLETA',
    'CICLA': 'BICICLETA',
    
    # Airplane variations
    'AVION': 'AVION',
    'AEREO': 'AVION',
    'VUELO': 'AVION',
    'TRANSPORTE AEREO': 'AVION',
    'LINEA AEREA': 'AVION',
    
    # Taxi/Ride sharing variations
    'TAXI': 'TAXI',
    'UBER': 'TAXI/PLATAFORMA',
    'DIDI': 'TAXI/PLATAFORMA',
    'CABIFY': 'TAXI/PLATAFORMA',
    'BEAT': 'TAXI/PLATAFORMA',
    'PLATAFORMA DIGITAL': 'TAXI/PLATAFORMA',
    'APLICACION': 'TAXI/PLATAFORMA',
    
    # Train variations
    'TREN': 'TREN',
    'FERROCARRIL': 'TREN',
    'METRO': 'TREN',
    'TRANSMILENIO': 'TRANSPORTE MASIVO',
    'BRT': 'TRANSPORTE MASIVO',
    'SISTEMA INTEGRADO': 'TRANSPORTE MASIVO',
    
    # Walking variations
    'CAMINANDO': 'A PIE',
    'PIE': 'A PIE',
    'A PIE': 'A PIE',
    'PEATONAL': 'A PIE',
    'CAMINATA': 'A PIE',
    
    # Boat/Water transport
    'BARCO': 'TRANSPORTE ACUATICO',
    'LANCHA': 'TRANSPORTE ACUATICO',
    'EMBARCACION': 'TRANSPORTE ACUATICO',
    'FLUVIAL': 'TRANSPORTE ACUATICO',
    'ACUATICO': 'TRANSPORTE ACUATICO',
    
    # Rental/Tour vehicles
    'VEHICULO ALQUILADO': 'VEHICULO ALQUILADO',
    'ALQUILER': 'VEHICULO ALQUILADO',
    'RENTAL': 'VEHICULO ALQUILADO',
    'RENT A CAR': 'VEHICULO ALQUILADO',
    'TOUR': 'TOUR/EXCURSION',
    'EXCURSION': 'TOUR/EXCURSION',
    'PAQUETE TURISTICO': 'TOUR/EXCURSION',
    'AGENCIA': 'TOUR/EXCURSION',
    
    # Multiple/Combined transport
    'MIXTO': 'TRANSPORTE MIXTO',
    'COMBINADO': 'TRANSPORTE MIXTO',
    'VARIOS': 'TRANSPORTE MIXTO',
    'MULTIPLE': 'TRANSPORTE MIXTO',
    
    # Other/Unspecified
    'OTRO': 'OTROS',
    'OTROS': 'OTROS',
    'OTRA': 'OTROS',
    'DIFERENTE': 'OTROS',
    'NS/NR': 'NO ESPECIFICA',
    'NO SABE': 'NO ESPECIFICA',
    'NO RESPONDE': 'NO ESPECIFICA',
    'SIN ESPECIFICAR': 'NO ESPECIFICA',
    'NO APLICA': 'NO ESPECIFICA'
}

In [12]:
def normalize_text(text):
    """
    Normalize text by removing accents, converting to uppercase, and cleaning special characters
    """
    if pd.isna(text) or text == 'NA':
        return text
    
    # Convert to string and strip whitespace
    text = str(text).strip()
    
    # Remove accents and normalize unicode characters
    text = unicodedata.normalize('NFD', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
    
    # Convert to uppercase for consistency
    text = text.upper()
    
    # Remove extra spaces and special characters (except essential ones)
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s-]', '', text)  # Keep only alphanumeric, spaces, and hyphens
    
    return text.strip()

def apply_city_mapping(normalized_city):
    """Apply manual mapping for known city variations"""
    if pd.isna(normalized_city) or normalized_city == 'NA':
        return normalized_city
    return city_mappings.get(normalized_city, normalized_city)

def apply_travel_motivation_mapping(normalized_motivation):
    """Apply manual mapping for known travel motivation variations"""
    if pd.isna(normalized_motivation) or normalized_motivation == 'NA':
        return normalized_motivation
    return travel_motivation_mappings.get(normalized_motivation, normalized_motivation)

def apply_transportation_mapping(normalized_transport):
    """Apply manual mapping for known transportation variations"""
    if pd.isna(normalized_transport) or normalized_transport == 'NA':
        return normalized_transport
    return transportation_mappings.get(normalized_transport, normalized_transport)

## Load and process

In [13]:
# Load dataset
df = pd.read_csv('Datos_de_las_Visitas_a_los_Puntos_de_Información_Turística_(PITS)_en_Boyacá_20250919.csv')

# Apply normalization to city columns
city_columns = ['CIUDAD DE PROCEDENCIA', 'CIUDAD DE DESTINO 1', 'CIUDAD DE DESTINO 2', 
                'CIUDAD DE DESTINO 3', 'CIUDAD DE DESTINO 4', 'CIUDAD DE DESTINO 5']

# Create normalized versions of city columns
for col in city_columns:
    df[f'{col}_NORMALIZED'] = df[col].apply(normalize_text)

# Apply manual mappings to normalized city columns
for col in city_columns:
    normalized_col = f'{col}_NORMALIZED'
    df[f'{col}_CLEAN'] = df[normalized_col].apply(apply_city_mapping)

# Apply normalization and mapping to travel motivation
df['MOTIVO_NORMALIZED'] = df['MOTIVO DEL VIAJE'].apply(normalize_text)
df['MOTIVO_CLEAN'] = df['MOTIVO_NORMALIZED'].apply(apply_travel_motivation_mapping)

# Apply normalization and mapping to transportation mode
df['TRANSPORTE_NORMALIZED'] = df['TIPO TRANSPORTE'].apply(normalize_text)
df['TRANSPORTE_CLEAN'] = df['TRANSPORTE_NORMALIZED'].apply(apply_transportation_mapping)

# Update working columns to use cleaned versions
df['CIUDAD_ORIGEN_CLEAN'] = df['CIUDAD DE PROCEDENCIA_CLEAN']


# Analize

In [ ]:

# Tourist Information Points (PIT) analysis
pit_counts = df['PUNTO DE INFORMACIÓN'].value_counts()

# Temporal analysis
## Monthly distribution
month_counts = df['MES'].value_counts().sort_index()

## Yearly distribution
year_counts = df['AÑO'].value_counts().sort_index()

# Cities analysis (using cleaned data)
## Origin cities analysis
origin_counts = df['CIUDAD_ORIGEN_CLEAN'].value_counts()

## Destination cities analysis
# Combine all cleaned destination columns
destination_cols_clean = ['CIUDAD DE DESTINO 1_CLEAN', 'CIUDAD DE DESTINO 2_CLEAN', 
                         'CIUDAD DE DESTINO 3_CLEAN', 'CIUDAD DE DESTINO 4_CLEAN', 
                         'CIUDAD DE DESTINO 5_CLEAN']
all_destinations = []
for col in destination_cols_clean:
    destinations = df[col].dropna()
    destinations = destinations[destinations != 'NA']
    all_destinations.extend(destinations.tolist())

destination_counts = pd.Series(all_destinations).value_counts()

### Calculate rate for most visited destinations
most_visited_rate = (destination_counts.head(5).sum() / len(all_destinations)) * 100

### Calculate popularity by number of destinations visited
df['num_destinations'] = 0
for col in destination_cols_clean:
    df['num_destinations'] += ((df[col].notna()) & (df[col] != 'NA')).astype(int)

destination_popularity = df['num_destinations'].value_counts().sort_index()

# Travel behavior analysis (using cleaned data)
## Travel motivation analysis
travel_reason = df['MOTIVO_CLEAN'].value_counts()
travel_reason_original = df['MOTIVO DEL VIAJE'].value_counts()

## Transportation mode analysis (using cleaned data)
transport = df['TRANSPORTE_CLEAN'].value_counts()
transport_original = df['TIPO TRANSPORTE'].value_counts()

## Accommodation type analysis
accommodation = df['TIPO ALOJAMIENTO'].value_counts()

## Trip duration analysis
duration_stats = df['TIEMPO DE PERMANENCIA EN DIAS'].describe()
duration_counts = df['TIEMPO DE PERMANENCIA EN DIAS'].value_counts().sort_index()

## Group size analysis (how many people travel together)
travelers_stats = df['CANTIDAD DE VIAJEROS'].describe()
travelers_counts = df['CANTIDAD DE VIAJEROS'].value_counts().sort_index()

# Additional demographic and economic analysis
## Budget range analysis
budget_counts = df['PRESUPUESTO'].value_counts()

## Customer satisfaction analysis
satisfaction_stats = df['RESULTADO SATISFACCIÓN'].describe()

## Gender distribution analysis
gender_counts = df['SEXO'].value_counts()

## Country of origin analysis
country_counts = df['PAIS DE PROCEDENCIA'].value_counts()

# Data quality report

# Interactive Visualizations

In [9]:
# 1. Tourist Information Points (PITs) - Total visitors by location
pit_data = df.groupby('PUNTO DE INFORMACIÓN')['CANTIDAD DE VIAJEROS'].sum().reset_index()
source_pit = ColumnDataSource(pit_data)

p1 = figure(x_range=pit_data['PUNTO DE INFORMACIÓN'], height=400, 
           title='Total Visitors by Tourist Information Point',
           tools='pan,wheel_zoom,box_zoom,reset,save', toolbar_location="above")

p1.vbar(x='PUNTO DE INFORMACIÓN', top='CANTIDAD DE VIAJEROS', width=0.8, source=source_pit,
        color=factor_cmap('PUNTO DE INFORMACIÓN', palette=Category20[len(pit_data)], 
                         factors=pit_data['PUNTO DE INFORMACIÓN'].tolist()))

p1.add_tools(HoverTool(tooltips=[('PIT', '@{PUNTO DE INFORMACIÓN}'), 
                                ('Total Visitors', '@{CANTIDAD DE VIAJEROS}')]))
p1.xaxis.major_label_orientation = 1.2
p1.title.text_font_size = "14pt"
show(p1)

# 2. Temporal Analysis - Monthly visit distribution
month_data = df['MES'].value_counts().sort_index().reset_index()
month_data.columns = ['Month', 'Count']
source_month = ColumnDataSource(month_data)

p2 = figure(x_range=[str(x) for x in month_data['Month']], height=350, 
           title='Monthly Visit Distribution',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p2.vbar(x='Month', top='Count', width=0.8, source=source_month, color='navy', alpha=0.7)
p2.add_tools(HoverTool(tooltips=[('Month', '@Month'), ('Visits', '@Count')]))
p2.title.text_font_size = "14pt"
show(p2)

# 3. Top origin cities (using cleaned data)
origin_data = df['CIUDAD_ORIGEN_CLEAN'].value_counts().head(10).reset_index()
origin_data.columns = ['City', 'Count']
source_origin = ColumnDataSource(origin_data)

p3 = figure(y_range=origin_data['City'], height=400, 
           title='Top 10 Origin Cities (Cleaned Data)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p3.hbar(y='City', right='Count', height=0.8, source=source_origin, 
        color='green', alpha=0.7)
p3.add_tools(HoverTool(tooltips=[('City', '@City'), ('Visitors', '@Count')]))
p3.title.text_font_size = "14pt"
show(p3)

# 4. Top destination cities (using cleaned data)
destination_data = destination_counts.head(10).reset_index()
destination_data.columns = ['City', 'Count']
source_dest = ColumnDataSource(destination_data)

p4 = figure(y_range=destination_data['City'], height=400, 
           title='Top 10 Destination Cities (Cleaned Data)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p4.hbar(y='City', right='Count', height=0.8, source=source_dest, 
        color='orange', alpha=0.7)
p4.add_tools(HoverTool(tooltips=[('City', '@City'), ('Visits', '@Count')]))
p4.title.text_font_size = "14pt"
show(p4)

# 5. Travel motivation analysis (using cleaned data)
reason_data = df['MOTIVO_CLEAN'].value_counts().reset_index()
reason_data.columns = ['Reason', 'Count']
source_reason = ColumnDataSource(reason_data)

p5 = figure(x_range=reason_data['Reason'], height=400, 
           title='Travel Motivation Distribution (Cleaned Data)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p5.vbar(x='Reason', top='Count', width=0.8, source=source_reason,
        color=factor_cmap('Reason', palette=Viridis256[:len(reason_data)], 
                         factors=reason_data['Reason'].tolist()))
p5.add_tools(HoverTool(tooltips=[('Reason', '@Reason'), ('Count', '@Count')]))
p5.xaxis.major_label_orientation = 1.2
p5.title.text_font_size = "14pt"
show(p5)

# 6. Transportation mode analysis (using cleaned data)
transport_data = df['TRANSPORTE_CLEAN'].value_counts().reset_index()
transport_data.columns = ['Transport', 'Count']
source_transport = ColumnDataSource(transport_data)

p6 = figure(x_range=transport_data['Transport'], height=400, 
           title='Transportation Mode Distribution (Cleaned Data)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p6.vbar(x='Transport', top='Count', width=0.8, source=source_transport, 
        color=factor_cmap('Transport', palette=Category20[:len(transport_data)], 
                         factors=transport_data['Transport'].tolist()), alpha=0.7)
p6.add_tools(HoverTool(tooltips=[('Transport', '@Transport'), ('Count', '@Count')]))
p6.xaxis.major_label_orientation = 1.2
p6.title.text_font_size = "14pt"
show(p6)

# 7. Trip duration analysis
duration_data = df['TIEMPO DE PERMANENCIA EN DIAS'].value_counts().sort_index().reset_index()
duration_data.columns = ['Days', 'Count']
source_duration = ColumnDataSource(duration_data)

p7 = figure(height=400, title='Trip Duration Distribution (Days)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p7.line(x='Days', y='Count', source=source_duration, line_width=3, color='purple')
p7.circle(x='Days', y='Count', source=source_duration, size=8, color='purple', alpha=0.7)
p7.add_tools(HoverTool(tooltips=[('Days', '@Days'), ('Count', '@Count')]))
p7.title.text_font_size = "14pt"
show(p7)

# 8. Group size analysis
travelers_data = df['CANTIDAD DE VIAJEROS'].value_counts().sort_index().reset_index()
travelers_data.columns = ['Group_Size', 'Count']
source_travelers = ColumnDataSource(travelers_data)

p8 = figure(height=400, title='Travel Group Size Distribution',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p8.vbar(x='Group_Size', top='Count', width=0.8, source=source_travelers, color='teal', alpha=0.7)
p8.add_tools(HoverTool(tooltips=[('Group Size', '@Group_Size'), ('Number of Groups', '@Count')]))
p8.title.text_font_size = "14pt"
show(p8)

# 9. Budget range analysis
budget_data = df['PRESUPUESTO'].value_counts().reset_index()
budget_data.columns = ['Budget', 'Count']
source_budget = ColumnDataSource(budget_data)

p9 = figure(x_range=budget_data['Budget'], height=400, 
           title='Budget Range Distribution',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p9.vbar(x='Budget', top='Count', width=0.8, source=source_budget, color='gold', alpha=0.7)
p9.add_tools(HoverTool(tooltips=[('Budget Range', '@Budget'), ('Count', '@Count')]))
p9.xaxis.major_label_orientation = 1.2
p9.title.text_font_size = "14pt"
show(p9)

# 10. Customer satisfaction analysis
satisfaction_data = df['RESULTADO SATISFACCIÓN'].value_counts().sort_index().reset_index()
satisfaction_data.columns = ['Satisfaction', 'Count']
source_satisfaction = ColumnDataSource(satisfaction_data)

p10 = figure(height=400, title='Customer Satisfaction Level Distribution',
            tools='pan,wheel_zoom,box_zoom,reset,save')

p10.vbar(x='Satisfaction', top='Count', width=0.6, source=source_satisfaction, 
         color='darkgreen', alpha=0.7)
p10.add_tools(HoverTool(tooltips=[('Satisfaction Level', '@Satisfaction'), ('Count', '@Count')]))
p10.title.text_font_size = "14pt"
show(p10)

# 11. Data Quality Comparison Visualization - Cities
comparison_data = pd.DataFrame({
    'Original_Count': len(df['CIUDAD DE PROCEDENCIA'].unique()),
    'Cleaned_Count': len(df['CIUDAD_ORIGEN_CLEAN'].unique()),
    'Reduction': len(df['CIUDAD DE PROCEDENCIA'].unique()) - len(df['CIUDAD_ORIGEN_CLEAN'].unique())
}, index=[0])

p11 = figure(x_range=['Original Cities', 'Cleaned Cities'], height=350, 
            title='Data Cleaning Impact: Number of Unique Origin Cities',
            tools='pan,wheel_zoom,box_zoom,reset,save')

cities_data = pd.DataFrame({
    'Category': ['Original Cities', 'Cleaned Cities'],
    'Count': [comparison_data['Original_Count'].iloc[0], comparison_data['Cleaned_Count'].iloc[0]]
})
source_cities = ColumnDataSource(cities_data)

p11.vbar(x='Category', top='Count', width=0.6, source=source_cities, 
         color=['red', 'green'], alpha=0.7)
p11.add_tools(HoverTool(tooltips=[('Category', '@Category'), ('Count', '@Count')]))
p11.title.text_font_size = "14pt"
show(p11)

# 12. Data Quality Comparison Visualization - Travel Motivations
motivation_comparison_data = pd.DataFrame({
    'Original_Count': len(df['MOTIVO DEL VIAJE'].unique()),
    'Cleaned_Count': len(df['MOTIVO_CLEAN'].unique()),
    'Reduction': len(df['MOTIVO DEL VIAJE'].unique()) - len(df['MOTIVO_CLEAN'].unique())
}, index=[0])

p12 = figure(x_range=['Original Motivations', 'Cleaned Motivations'], height=350, 
            title='Data Cleaning Impact: Number of Unique Travel Motivations',
            tools='pan,wheel_zoom,box_zoom,reset,save')

motivations_data = pd.DataFrame({
    'Category': ['Original Motivations', 'Cleaned Motivations'],
    'Count': [motivation_comparison_data['Original_Count'].iloc[0], motivation_comparison_data['Cleaned_Count'].iloc[0]]
})
source_motivations = ColumnDataSource(motivations_data)

p12.vbar(x='Category', top='Count', width=0.6, source=source_motivations, 
         color=['red', 'green'], alpha=0.7)
p12.add_tools(HoverTool(tooltips=[('Category', '@Category'), ('Count', '@Count')]))
p12.title.text_font_size = "14pt"
show(p12)

# 13. Data Quality Comparison Visualization - Transportation Modes
transport_comparison_data = pd.DataFrame({
    'Original_Count': len(df['TIPO TRANSPORTE'].unique()),
    'Cleaned_Count': len(df['TRANSPORTE_CLEAN'].unique()),
    'Reduction': len(df['TIPO TRANSPORTE'].unique()) - len(df['TRANSPORTE_CLEAN'].unique())
}, index=[0])

p13 = figure(x_range=['Original Transport', 'Cleaned Transport'], height=350, 
            title='Data Cleaning Impact: Number of Unique Transportation Modes',
            tools='pan,wheel_zoom,box_zoom,reset,save')

transport_comp_data = pd.DataFrame({
    'Category': ['Original Transport', 'Cleaned Transport'],
    'Count': [transport_comparison_data['Original_Count'].iloc[0], transport_comparison_data['Cleaned_Count'].iloc[0]]
})
source_transport_comp = ColumnDataSource(transport_comp_data)

p13.vbar(x='Category', top='Count', width=0.6, source=source_transport_comp, 
         color=['red', 'green'], alpha=0.7)
p13.add_tools(HoverTool(tooltips=[('Category', '@Category'), ('Count', '@Count')]))
p13.title.text_font_size = "14pt"
show(p13)

# 14. Side-by-side comparison: Original vs Cleaned Transportation Modes
# Original transport modes (top 8)
original_transport = df['TIPO TRANSPORTE'].value_counts().head(8).reset_index()
original_transport.columns = ['Transport', 'Count']
original_transport['Type'] = 'Original'

# Cleaned transport modes (top 8)
cleaned_transport = df['TRANSPORTE_CLEAN'].value_counts().head(8).reset_index()
cleaned_transport.columns = ['Transport', 'Count']
cleaned_transport['Type'] = 'Cleaned'

# Create combined data for comparison
combined_transport = pd.concat([original_transport, cleaned_transport], ignore_index=True)

p14 = figure(x_range=combined_transport['Transport'].unique(), height=500,
            title='Transportation Modes: Original vs Cleaned Data Comparison',
            tools='pan,wheel_zoom,box_zoom,reset,save')

# Create grouped bars
from bokeh.transform import dodge

p14.vbar(x=dodge('Transport', -0.2, range=p14.x_range), top='Count', width=0.35, 
         source=ColumnDataSource(original_transport), color='red', alpha=0.7, legend_label='Original')

p14.vbar(x=dodge('Transport', 0.2, range=p14.x_range), top='Count', width=0.35,
         source=ColumnDataSource(cleaned_transport), color='green', alpha=0.7, legend_label='Cleaned')

p14.add_tools(HoverTool(tooltips=[('Transport', '@Transport'), ('Count', '@Count'), ('Type', '@Type')]))
p14.xaxis.major_label_orientation = 1.2
p14.legend.location = "top_right"
p14.title.text_font_size = "14pt"
show(p14)

# 15. Side-by-side comparison: Original vs Cleaned Travel Motivations  
# Original motivations (top 8)
original_motives = df['MOTIVO DEL VIAJE'].value_counts().head(8).reset_index()
original_motives.columns = ['Motivation', 'Count']
original_motives['Type'] = 'Original'

# Cleaned motivations (top 8)
cleaned_motives = df['MOTIVO_CLEAN'].value_counts().head(8).reset_index()
cleaned_motives.columns = ['Motivation', 'Count']
cleaned_motives['Type'] = 'Cleaned'

# Create combined data for comparison
combined_motives = pd.concat([original_motives, cleaned_motives], ignore_index=True)

p15 = figure(x_range=combined_motives['Motivation'].unique(), height=500,
            title='Travel Motivations: Original vs Cleaned Data Comparison',
            tools='pan,wheel_zoom,box_zoom,reset,save')

p15.vbar(x=dodge('Motivation', -0.2, range=p15.x_range), top='Count', width=0.35, 
         source=ColumnDataSource(original_motives), color='red', alpha=0.7, legend_label='Original')

p15.vbar(x=dodge('Motivation', 0.2, range=p15.x_range), top='Count', width=0.35,
         source=ColumnDataSource(cleaned_motives), color='green', alpha=0.7, legend_label='Cleaned')

p15.add_tools(HoverTool(tooltips=[('Motivation', '@Motivation'), ('Count', '@Count'), ('Type', '@Type')]))
p15.xaxis.major_label_orientation = 1.2
p15.legend.location = "top_right"
p15.title.text_font_size = "14pt"
show(p15)

TypeError: unhashable type: 'slice'

In [ ]:
# ADVANCED VISUALIZATIONS - COMBINED ANALYSIS

# 16. Heatmap: PIT vs Month cross-analysis
heatmap_data = df.groupby(['PUNTO DE INFORMACIÓN', 'MES']).size().reset_index(name='count')
pit_list = sorted(df['PUNTO DE INFORMACIÓN'].unique())
month_list = sorted(df['MES'].unique())

# Create complete matrix for heatmap
heatmap_matrix = []
for pit in pit_list:
    for month in month_list:
        count = heatmap_data[(heatmap_data['PUNTO DE INFORMACIÓN'] == pit) & 
                            (heatmap_data['MES'] == month)]['count'].values
        count = count[0] if len(count) > 0 else 0
        heatmap_matrix.append({'pit': pit, 'month': str(month), 'count': count})

source_heatmap = ColumnDataSource(pd.DataFrame(heatmap_matrix))

# Define color palette for heatmap
colors = ["#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6", "#2171b5", "#08519c", "#08306b"]
mapper = LinearColorMapper(palette=colors, low=0, high=max([d['count'] for d in heatmap_matrix]))

p16 = figure(title="Visit Intensity by PIT and Month", x_range=[str(m) for m in month_list], y_range=pit_list,
            x_axis_location="above", width=600, height=400,
            tools="hover,save,pan,box_zoom,reset,wheel_zoom", toolbar_location='below')

p16.rect(x="month", y="pit", width=1, height=1, source=source_heatmap,
         fill_color=transform('count', mapper), line_color=None)

color_bar = ColorBar(color_mapper=mapper, width=8, location=(0,0))
p16.add_layout(color_bar, 'right')

p16.add_tools(HoverTool(tooltips=[('PIT', '@pit'), ('Month', '@month'), ('Visits', '@count')]))
show(p16)

# 17. Scatter plot: Trip Duration vs Group Size (colored by satisfaction)
# Filter valid data points
scatter_data = df[(df['TIEMPO DE PERMANENCIA EN DIAS'] > 0) & 
                 (df['CANTIDAD DE VIAJEROS'] > 0) & 
                 (df['RESULTADO SATISFACCIÓN'] > 0)].copy()

source_scatter = ColumnDataSource(scatter_data)

p17 = figure(title="Trip Duration vs Group Size (Colored by Satisfaction)", 
            x_axis_label='Trip Duration (days)', 
            y_axis_label='Group Size',
            width=700, height=500,
            tools="pan,wheel_zoom,box_zoom,reset,save")

# Create color mapping based on satisfaction level
satisfaction_factors = [str(x) for x in sorted(scatter_data['RESULTADO SATISFACCIÓN'].unique())]
satisfaction_colors = factor_cmap('RESULTADO SATISFACCIÓN', 
                                 palette=Viridis256[::30], 
                                 factors=satisfaction_factors)

p17.circle('TIEMPO DE PERMANENCIA EN DIAS', 'CANTIDAD DE VIAJEROS', 
          source=source_scatter, size=8, alpha=0.7, color=satisfaction_colors)

p17.add_tools(HoverTool(tooltips=[('Duration', '@{TIEMPO DE PERMANENCIA EN DIAS} days'),
                                 ('Group Size', '@{CANTIDAD DE VIAJEROS}'),
                                 ('Satisfaction', '@{RESULTADO SATISFACCIÓN}'),
                                 ('PIT', '@{PUNTO DE INFORMACIÓN}')]))
show(p17)

# 18. Multi-destination travel pattern analysis
multi_dest_data = df['num_destinations'].value_counts().sort_index().reset_index()
multi_dest_data.columns = ['Num_Destinations', 'Count']
source_multi = ColumnDataSource(multi_dest_data)

p18 = figure(title="Distribution by Number of Destinations Visited",
            x_axis_label='Number of Destinations', y_axis_label='Number of Tourists',
            width=600, height=400,
            tools="pan,wheel_zoom,box_zoom,reset,save")

p18.vbar(x='Num_Destinations', top='Count', width=0.8, source=source_multi, 
         color='coral', alpha=0.8)

p18.add_tools(HoverTool(tooltips=[('Destinations', '@Num_Destinations'), 
                                 ('Tourists', '@Count')]))
show(p18)

# 19. Gender vs Travel Motivation cross-analysis (using cleaned motivations)
gender_reason = df.groupby(['SEXO', 'MOTIVO_CLEAN']).size().reset_index(name='count')
source_gender = ColumnDataSource(gender_reason)

motivos = sorted(df['MOTIVO_CLEAN'].unique())
p19 = figure(x_range=motivos, height=400, 
            title="Travel Motivation by Gender (Cleaned Data)",
            tools="pan,wheel_zoom,box_zoom,reset,save")

# Separate data by gender
masculino_data = gender_reason[gender_reason['SEXO'] == 'Masculino']
femenino_data = gender_reason[gender_reason['SEXO'] == 'Femenino']

# Create grouped bar chart
p19.vbar(x='MOTIVO_CLEAN', top='count', width=0.4, 
         source=ColumnDataSource(masculino_data), 
         color='blue', alpha=0.7, legend_label='Male')

# Adjust position for female bars  
for i, motivo in enumerate(motivos):
    fem_count = femenino_data[femenino_data['MOTIVO_CLEAN'] == motivo]['count'].values
    if len(fem_count) > 0:
        p19.vbar(x=i+0.4, top=fem_count[0], width=0.4, 
                color='pink', alpha=0.7)

p19.add_tools(HoverTool(tooltips=[('Motivation', '@{MOTIVO_CLEAN}'), 
                                 ('Count', '@count'),
                                 ('Gender', '@SEXO')]))
p19.xaxis.major_label_orientation = 1.2
p19.legend.location = "top_right"
show(p19)

# 20. Heatmap: Travel Motivation vs Transportation Mode (using cleaned data)
motivation_transport_data = df.groupby(['MOTIVO_CLEAN', 'TRANSPORTE_CLEAN']).size().reset_index(name='count')
motivation_list = sorted(df['MOTIVO_CLEAN'].unique())
transport_list = sorted(df['TRANSPORTE_CLEAN'].unique())

# Create complete matrix for motivation-transport heatmap
motivation_transport_matrix = []
for motivation in motivation_list:
    for transport in transport_list:
        count = motivation_transport_data[(motivation_transport_data['MOTIVO_CLEAN'] == motivation) & 
                                         (motivation_transport_data['TRANSPORTE_CLEAN'] == transport)]['count'].values
        count = count[0] if len(count) > 0 else 0
        motivation_transport_matrix.append({'motivation': motivation, 'transport': transport, 'count': count})

source_motivation_transport = ColumnDataSource(pd.DataFrame(motivation_transport_matrix))

# Define color palette for heatmap
colors_motivation_transport = ["#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6", "#2171b5", "#08519c", "#08306b"]
mapper_motivation_transport = LinearColorMapper(palette=colors_motivation_transport, low=0, 
                                               high=max([d['count'] for d in motivation_transport_matrix]))

p20 = figure(title="Travel Patterns: Motivation vs Transportation Mode (Cleaned Data)", 
            x_range=transport_list, y_range=motivation_list,
            x_axis_location="above", width=800, height=500,
            tools="hover,save,pan,box_zoom,reset,wheel_zoom", toolbar_location='below')

p20.rect(x="transport", y="motivation", width=1, height=1, source=source_motivation_transport,
         fill_color=transform('count', mapper_motivation_transport), line_color=None)

color_bar_motivation_transport = ColorBar(color_mapper=mapper_motivation_transport, width=8, location=(0,0))
p20.add_layout(color_bar_motivation_transport, 'right')

p20.add_tools(HoverTool(tooltips=[('Motivation', '@motivation'), ('Transport', '@transport'), ('Visits', '@count')]))
p20.xaxis.major_label_orientation = 1.2
show(p20)

# 21. Heatmap: Travel Motivation vs PIT cross-analysis (using cleaned data)
motivation_pit_data = df.groupby(['MOTIVO_CLEAN', 'PUNTO DE INFORMACIÓN']).size().reset_index(name='count')
pit_list_full = sorted(df['PUNTO DE INFORMACIÓN'].unique())

# Create complete matrix for motivation-pit heatmap
motivation_pit_matrix = []
for motivation in motivation_list:
    for pit in pit_list_full:
        count = motivation_pit_data[(motivation_pit_data['MOTIVO_CLEAN'] == motivation) & 
                                   (motivation_pit_data['PUNTO DE INFORMACIÓN'] == pit)]['count'].values
        count = count[0] if len(count) > 0 else 0
        motivation_pit_matrix.append({'motivation': motivation, 'pit': pit, 'count': count})

source_motivation_heatmap = ColumnDataSource(pd.DataFrame(motivation_pit_matrix))

# Define color palette for heatmap
colors_motivation = ["#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6", "#2171b5", "#08519c", "#08306b"]
mapper_motivation = LinearColorMapper(palette=colors_motivation, low=0, 
                                    high=max([d['count'] for d in motivation_pit_matrix]))

p21 = figure(title="Visit Patterns: Travel Motivation vs Tourist Information Points", 
            x_range=pit_list_full, y_range=motivation_list,
            x_axis_location="above", width=800, height=500,
            tools="hover,save,pan,box_zoom,reset,wheel_zoom", toolbar_location='below')

p21.rect(x="pit", y="motivation", width=1, height=1, source=source_motivation_heatmap,
         fill_color=transform('count', mapper_motivation), line_color=None)

color_bar_motivation = ColorBar(color_mapper=mapper_motivation, width=8, location=(0,0))
p21.add_layout(color_bar_motivation, 'right')

p21.add_tools(HoverTool(tooltips=[('Motivation', '@motivation'), ('PIT', '@pit'), ('Visits', '@count')]))
p21.xaxis.major_label_orientation = 1.2
show(p21)

# 22. Transportation Mode vs PIT cross-analysis (using cleaned data)
transport_pit_data = df.groupby(['TRANSPORTE_CLEAN', 'PUNTO DE INFORMACIÓN']).size().reset_index(name='count')

# Create complete matrix for transport-pit heatmap
transport_pit_matrix = []
for transport in transport_list:
    for pit in pit_list_full:
        count = transport_pit_data[(transport_pit_data['TRANSPORTE_CLEAN'] == transport) & 
                                  (transport_pit_data['PUNTO DE INFORMACIÓN'] == pit)]['count'].values
        count = count[0] if len(count) > 0 else 0
        transport_pit_matrix.append({'transport': transport, 'pit': pit, 'count': count})

source_transport_heatmap = ColumnDataSource(pd.DataFrame(transport_pit_matrix))

# Define color palette for heatmap
colors_transport = ["#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6", "#2171b5", "#08519c", "#08306b"]
mapper_transport = LinearColorMapper(palette=colors_transport, low=0, 
                                   high=max([d['count'] for d in transport_pit_matrix]))

p22 = figure(title="Transportation Patterns: Transport Mode vs Tourist Information Points", 
            x_range=pit_list_full, y_range=transport_list,
            x_axis_location="above", width=800, height=500,
            tools="hover,save,pan,box_zoom,reset,wheel_zoom", toolbar_location='below')

p22.rect(x="pit", y="transport", width=1, height=1, source=source_transport_heatmap,
         fill_color=transform('count', mapper_transport), line_color=None)

color_bar_transport = ColorBar(color_mapper=mapper_transport, width=8, location=(0,0))
p22.add_layout(color_bar_transport, 'right')

p22.add_tools(HoverTool(tooltips=[('Transport', '@transport'), ('PIT', '@pit'), ('Visits', '@count')]))
p22.xaxis.major_label_orientation = 1.2
show(p22)

# 23. Budget vs Transportation Mode analysis (using cleaned data)
budget_transport = df.groupby(['PRESUPUESTO', 'TRANSPORTE_CLEAN']).size().reset_index(name='count')
budget_list = sorted(df['PRESUPUESTO'].unique())

p23 = figure(x_range=budget_list, height=400,
            title="Budget Range vs Transportation Mode (Cleaned Data)",
            tools="pan,wheel_zoom,box_zoom,reset,save")

# Create stacked bars by transportation mode
transports_for_budget = df['TRANSPORTE_CLEAN'].unique()[:5]  # Top 5 transportation modes
colors_budget_transport = ['red', 'green', 'blue', 'orange', 'purple']

bottom = [0] * len(budget_list)
for i, transport in enumerate(transports_for_budget):
    counts = []
    for budget in budget_list:
        count = budget_transport[(budget_transport['PRESUPUESTO'] == budget) & 
                                (budget_transport['TRANSPORTE_CLEAN'] == transport)]['count'].values
        counts.append(count[0] if len(count) > 0 else 0)
    
    p23.vbar(x=budget_list, top=counts, bottom=bottom, width=0.8, 
            color=colors_budget_transport[i], alpha=0.7, legend_label=transport)
    
    # Update bottom for stacking
    bottom = [b + c for b, c in zip(bottom, counts)]

p23.add_tools(HoverTool(tooltips=[('Budget', '@x'), ('Count', '@top')]))
p23.xaxis.major_label_orientation = 1.2
p23.legend.location = "top_right"
p23.title.text_font_size = "14pt"
show(p23)